__Final Project__

# Data Collection Process

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import random
import os
from sklearn.preprocessing import MinMaxScaler
from spotipy import Spotify  
# from dotenv import load_dotenv

## Loading Client Information:

In [58]:


client_id = '5477354bc87a4af1b2b8b10e23998f8d'
client_secret = 'a7af1ca6c73c4aff944122081f208e16'
redirect_uri = 'info-1998-login://callback'
# scopes = "user-read-playback-state,user-modify-playback-state,streaming"

sp_oauth = SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)  

# access_token = sp_oauth.get_cached_token()  
# refresh_token = sp_oauth.refresh_access_token()  

sp = Spotify(auth_manager=sp_oauth)  

## Collecting Song Data:

In [62]:
def get_random_artists(limit=20, min_popularity=1, 
                                max_popularity=100):
    artist_list = []

    search_queries = [
        'cu', 'vu', 'bu', 'wu', 'ru', 
        'pu', 'qu', 'su', 'lu', 'nu']

    for query in search_queries:
        results = sp.search(q=f'artist:{query}', 
                            type='artist', market='US', 
                            limit=50, 
                            offset=random.randint(0, 950))
        for artist in results['artists']['items']:
            if min_popularity <= artist['popularity'] \
                <= max_popularity:
                artist_list.append(artist['name'])
                if len(artist_list) >= limit:
                    return artist_list[:limit]

    return artist_list[:limit]


# Ensure the client_id and client_secret are correct
# Re-authenticate with Spotify
sp_oauth = SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = Spotify(auth_manager=sp_oauth)

# Fetch random artists
artists = get_random_artists()
print(artists)


SpotifyOauthError: error: invalid_client, error_description: Invalid client

## Data Cleaning

//

# Research Question

//

## Model 1

## Visualization for Model 1

## Model 2

## Visualization for Model 2

# EXPLAIN EACH SECTION!!!